In [19]:
# 构建一个非常简单的数据集
sentences = ["我 喜欢 玩具 ", "我 爱 爸爸 ", "我 讨厌 挨打 "]
# 构建词汇表连接所有句子，用空格分隔，然后用set去重，最后转换为list
word_list = list(set("".join(sentences).split()))
# 构建词典
word_to_idx = {word: i for i, word in enumerate(word_list)}
# 构建逆词典
idx_to_word = {i: word for i, word in enumerate(word_list)}
voc_size = len(word_list)  # 词汇表的大小
print("词汇表: ", word_to_idx)
print("词汇表大小：", voc_size)

词汇表:  {'我': 0, '玩具': 1, '喜欢': 2, '爸爸': 3, '讨厌': 4, '爱': 5, '挨打': 6}
词汇表大小： 7


In [20]:
# 构建批处理数据
import torch
import random
batch_size = 2
def make_batch():
    input_batch = []  # 输入数据
    target_batch = []  # 输出数据
    selected_sentences = random.sample(sentences, batch_size)  # 随机选择batch_size个句子
    for sen in selected_sentences: # 遍历每个句子
        word = sen.split()  # 分词
        # 将除最后一个词以外的词作为输入，最后一个词作为输出
        input = [word_to_idx[w] for w in word[:-1]]
        target = word_to_idx[word[-1]]
        input_batch.append(input)  # 添加到输入数据
        target_batch.append(target)  # 添加到输出数据
    input_batch = torch.LongTensor(input_batch)  # 转换为LongTensor
    target_batch = torch.LongTensor(target_batch)  # 转换为LongTensor
    return input_batch, target_batch

input_batch, target_batch = make_batch()  # 生成批处理数据
print("输入批处理数据: ", input_batch)  # 打印输入数据
# 将输入批数据中的每个索引值转化为对应的原始词
input_words = []
for input_idx in input_batch:
    input_words.append([idx_to_word[idx.item()] for idx in input_idx])
print("输入批处理数据对应的原始词: ", input_words)  # 打印输入词
print("目标批处理数据：", target_batch)  # 打印输出数据
# 将输出批数据中的每个索引值转化为对应的原始词
target_words = [idx_to_word[idx.item()] for idx in target_batch]
print("目标批处理数据对应的原始词: ", target_words)  # 打印输出词

输入批处理数据:  tensor([[0, 5],
        [0, 2]])
输入批处理数据对应的原始词:  [['我', '爱'], ['我', '喜欢']]
目标批处理数据： tensor([3, 1])
目标批处理数据对应的原始词:  ['爸爸', '玩具']


In [21]:
n_step = 2  # 时间步数，表示每个输入序列的长度，即上下文长度
n_hidden = 2  # 隐藏层的大小
embedding_size = 2  # 词嵌入的大小

In [22]:
import torch.nn as nn  # 导入神经网络模块
# 定义神经概率语言模型
class NPLM(nn.Module):
    def __init__(self):
        super(NPLM, self).__init__()
        self.C = nn.Embedding(voc_size, embedding_size) # 词嵌入层
        # 第一个线性层，其输入大小为n_step * embedding_size，输出大小为n_hidden
        self.linerar1 = nn.Linear(n_step * embedding_size, n_hidden)
        # 第二个线性层，其输入大小为 n_hidden，输出大小为voc_size，即词汇表的大小
        self.linerar2 = nn.Linear(n_hidden, voc_size)
    def forward(self, X):
        # 输入张量的形状为(batch_size, n_step)
        X = self.C(X)  # X通过词嵌入层，形状变为(batch_size, n_step, embedding_size)
        X = X.view(-1, n_step * embedding_size) # X形状变为(batch_size, n_step * embedding_size)
        hidden = torch.tanh(self.linerar1(X))  # X通过第一个线性层，然后通过tanh激活函数，形状变为(batch_size, n_hidden)
        output = self.linerar2(hidden)  # hidden通过第二个线性层，形状变为(batch_size, voc_size)
        return output
        

In [23]:
model = NPLM()  # 实例化模型
print("模型结构：", model)  # 打印模型结构

模型结构： NPLM(
  (C): Embedding(7, 2)
  (linerar1): Linear(in_features=4, out_features=2, bias=True)
  (linerar2): Linear(in_features=2, out_features=7, bias=True)
)


In [24]:
import torch.optim as optim  # 导入优化器
criterion = nn.CrossEntropyLoss()  # 定义损失函数
optimizer = optim.Adam(model.parameters(), lr=0.1)  # 定义优化器
# 训练模型
for epoch in range(5000): # 训练5000次
    optimizer.zero_grad()  # 梯度清零
    input_batch, target_batch = make_batch()  # 生成批处理数据
    output = model(input_batch)  # 模型输出
    loss = criterion(output, target_batch)  # 计算损失
    if (epoch + 1) % 1000 == 0:  # 每1000次打印一次损失
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()  # 反向传播
    optimizer.step()  # 更新参数

Epoch: 1000 cost = 0.000610
Epoch: 2000 cost = 0.000224
Epoch: 3000 cost = 0.000100
Epoch: 4000 cost = 0.000054
Epoch: 5000 cost = 0.000029


In [29]:
# 进行预测
input_strs = [['我', '喜欢'], ['我', '讨厌']]  # 需要预测的输入序列
# 将输入序列转化为索引序列
input_batch = torch.LongTensor([[word_to_idx[word] for word in input_str] for input_str in input_strs])
# 进行预测
predict = model(input_batch).data.max(1)[1]
# 将预测结果转化为对应的词
predict_words = [idx_to_word[idx.item()] for idx in predict]
for input_seq, pred in zip(input_strs, predict_words):
    print(input_seq, "->", pred)

['我', '喜欢'] -> 玩具
['我', '讨厌'] -> 挨打
